In [1]:
import pandas as pd

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup

import time

In [2]:
driver = webdriver.Chrome(executable_path='C:/Users/17178/Downloads/chromedriver_win32/chromedriver.exe')
driver.get('https://www.carmax.com/cars/used/sedans')
content = driver.page_source
soup = BeautifulSoup(content)

In [3]:
#remove location filter to capture nationwide availability
remove_filter = driver.find_element_by_xpath("/html/body/main/div[1]/div/aside/div/div[1]/div[3]/div/button[1]/span/span[1]")
remove_filter.click()

#show manufacturer options
make_filter = driver.find_element_by_xpath("/html/body/main/div[1]/div/aside/div/div[2]/div/div[1]/div/div/div[1]/div[3]/button")
make_filter.click()

#read in data by manufacturer
for i in range(1,30):
    time.sleep(5)
    
    manufacturer_filter = driver.find_element_by_xpath(f"/html/body/main/div[1]/div/aside/div/div[2]/div/div[1]/div/div/div[2]/div/div/div[2]/div[1]/div/ul/li[{i}]")
    manufacturer_filter.click()

    time.sleep(5)

    while True:
        try:
            showmore = driver.find_element_by_xpath("/html/body/main/div[1]/div/section/section[1]/div[2]/div[3]/div/a")
            showmore.click()
            time.sleep(30)
        except NoSuchElementException:
            break

    content = driver.page_source
    soup = BeautifulSoup(content)

    results_year_make = []
    results_model_trim = []
    results_miles  = []
    results_price = []
    results_state = []

    for element in soup.findAll('article', {'class': 'car-tile'}):
        for year_make in element.findAll('span', {'class': 'year-make'}):
            results_year_make.append(year_make.contents[0])
        for model_trim in element.findAll('span', {'class': 'model-trim'}):
            results_model_trim.append(model_trim.contents[0])
        for price in element.findAll('span', {'class': 'price'}):
            results_price.append(price.contents[0])
        for state in element.findAll('span', {'class': 'transfer'}):
            results_state.append(state.contents[0][-2:])
        for miles in element.findAll('span', {'class': 'miles'}):
            results_miles.append(miles.contents[0])

    df_temp = pd.DataFrame(
                {'year_make': results_year_make,
                 'model_trim': results_model_trim,
                 'price': results_price,
                 'state': results_state,
                 'miles': results_miles,
                 'source': 'carmax',
                 'type': 'sedan'
                })
  
    if i == 1:
        df = df_temp
    else:
        df = pd.merge(df, df_temp, how='outer')
        del df_temp

    manufacturer_filter_off = driver.find_element_by_xpath("/html/body/main/div[1]/div/aside/div/div[1]/div[3]/div/button[1]/span/span[1]")
    manufacturer_filter_off.click()
    time.sleep(5)

df

,year_make,model_trim,price,state,miles,source,type
0,2014 Acura,TSX Special Edition,"$19,998*",MD,54K mi,carmax,sedan
1,2019 Acura,TLX,"$28,998*",MD,26K mi,carmax,sedan
2,2019 Acura,TLX,"$28,998*",MD,26K mi,carmax,sedan
3,2015 Acura,TLX,"$22,998*",MD,40K mi,carmax,sedan
4,2019 Acura,TLX,"$29,998*",MD,14K mi,carmax,sedan
...,...,...,...,...,...,...,...
12412,2016 Volkswagen,GTI SE,"$25,998*",CA,10K mi,carmax,sedan
12413,2016 Volkswagen,Golf SE,"$18,998*",NV,46K mi,carmax,sedan
12414,2019 Volkswagen,Passat SE R-Line,"$22,998*",TX,47K mi,carmax,sedan
12415,2017 Volkswagen,Jetta SE,"$18,998*",AZ,25K mi,carmax,sedan


In [5]:
#convert miles to numeric value
df['miles'] = df['miles'].str.extract('([0-9]+)').astype(float)*1000

#convert price to numeric value
df['price'] = df['price'].replace({'\$':''}, regex=True)
df['price'] = df['price'].replace({'\*':''}, regex=True)
df['price'] = df['price'].replace({'\,':''}, regex=True)
df['price'] = df['price'].astype(int)

In [6]:
df.isnull().sum()

year_make     0
model_trim    0
price         0
state         0
miles         0
source        0
type          0
dtype: int64

In [7]:
df.to_csv('C:/Users/17178/Documents/DS Projects/Nitorum Case/Data/carmax_sedan.csv')